In [3]:
#microphone check one two what is this!

In [49]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from statsmodels.tsa.statespace.sarimax import SARIMAX #pip install statsmodels
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

#from thefuzz import process

In [46]:
'''tube_perf = pd.read_excel("tfl-tube-performance.xlsx")

entry_exit = pd.read_excel("multi-year-station-entry-and-exit-figures.xlsx", sheet_name=None)

df_list = []
for year, data in entry_exit.items():
    data['year'] = year
    df_list.append(data)
entry_exit_df = pd.concat(df_list, ignore_index=True)'''
'''df_list = []
for sheet, data in entry_exit.items():
    year = re.findall(r'\d{4}', sheet)
    data['year'] = int(year[0]) if year else None
    df_list.append(data)

entry_exit_df = pd.concat(df_list, ignore_index=True)'''
'''tube_perf.columns = tube_perf.columns.str.strip().str.lower().str.replace(" ", "_")
entry_exit_df.columns = entry_exit_df.columns.str.strip().str.lower().str.replace(" ", "_")'''
'''tube_perf = tube_perf[['name', 'london_underground_performance_data']].rename(columns={'name': 'station_name'})
entry_exit_df = entry_exit_df[['station', 'local_authority', 'zone(s)[†]', 'year', 'usage[5]']]

#fuzzy matching
station_names_perf = tube_perf['station_name'].dropna().unique()
entry_exit_df['matched_station'] = entry_exit_df['station'].apply(
    lambda x: process.extractOne(x, station_names_perf)[0] if isinstance(x, str) else None)'''
'''#merging datasets
merged_df = pd.merge(
    entry_exit_df,
    tube_perf,
    left_on='matched_station',
    right_on='station_name',
    how='left')

print("Merged shape:", merged_df.shape)
print(merged_df.head())'''

'#merging datasets\nmerged_df = pd.merge(\n    entry_exit_df,\n    tube_perf,\n    left_on=\'matched_station\',\n    right_on=\'station_name\',\n    how=\'left\')\n\nprint("Merged shape:", merged_df.shape)\nprint(merged_df.head())'

In [47]:
import pandas as pd, re
from pprint import pprint
 
xl = pd.ExcelFile("multi-year-station-entry-and-exit-figures.xlsx")
 
def norm(s): 
    import re
    s = str(s).lower().strip()
    s = re.sub(r"[^a-z0-9]+", " ", s)
    return re.sub(r"\s+", " ", s).strip()
 
def find_header_row(xl, sheet, scan_rows=60):
    raw = xl.parse(sheet, header=None)
    for i in range(min(scan_rows, len(raw))):
        row = [norm(v) for v in raw.iloc[i].tolist()]
        if any("station" in v for v in row) or (any("entr" in v for v in row) and any("exit" in v for v in row)):
            return i
    return None
 
s = "2017 Entry & Exit"
hdr = find_header_row(xl, s)
df = xl.parse(s, header=hdr).dropna(how="all").dropna(axis=1, how="all")
pprint([norm(c) for c in df.columns])

['counts 2017 annual entries exits',
 'unnamed 1',
 'unnamed 2',
 'unnamed 3',
 'unnamed 4',
 'unnamed 5',
 'unnamed 6',
 'unnamed 7',
 'unnamed 8',
 'unnamed 9',
 'unnamed 10']


In [53]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
 
# ==== EDIT THESE PATHS IF NEEDED ============================================
RAW_STATION_XLSX   = "multi-year-station-entry-and-exit-figures.xlsx"
RAW_STATIONS_CSV   = "TfL_stations.csv"
RAW_PERF_XLSX      = "tfl-tube-performance.xlsx"
 
OUT_DIR            = "data/processed"
OUT_STATION_FLOW   = f"{OUT_DIR}/station_flow_clean_basic.csv"
OUT_STATIONS_CLEAN = f"{OUT_DIR}/TfL_stations_clean.csv"
OUT_PERF_CLEAN     = f"{OUT_DIR}/tube_performance_clean_basic.csv"
 
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)
 
# ==== A) CLEAN STATION ENTRY/EXIT WORKBOOK (1 CSV) ==========================
def clean_station_workbook(xlsx_path: str) -> pd.DataFrame:
    """
    Super-basic approach for messy, multi-sheet TfL station workbook.
    - Reads each 'Entry & Exit' sheet
    - Uses first column as station names
    - Picks the 'biggest' numeric column as annual entries+exits
    - Converts millions -> absolute if needed (simple heuristic)
    - Adds 'year' from sheet name
    """
    xl = pd.ExcelFile(xlsx_path)
    frames = []
 
    target_sheets = [s for s in xl.sheet_names if ("entry" in s.lower() and "exit" in s.lower())]
    for s in target_sheets:
        # raw read, drop empty rows/cols
        raw = xl.parse(s, header=None)
        raw = raw.dropna(how="all").dropna(axis=1, how="all")
        if raw.empty:
            continue
 
        # assume first column holds station names (common in these books)
        df = raw.copy()
        df.columns = [f"col{i}" for i in range(df.shape[1])]
 
        # coerce non-first columns to numeric when possible
        for c in df.columns[1:]:
            df[c] = pd.to_numeric(df[c], errors="coerce")
 
        # choose "total" as numeric column with highest 90th percentile
        num_cols = [c for c in df.columns[1:] if np.issubdtype(df[c].dtype, np.number)]
        if not num_cols:
            continue
        q90 = {c: df[c].quantile(0.90) for c in num_cols}
        total_col = max(q90, key=lambda k: (q90[k] if pd.notna(q90[k]) else -np.inf))
 
        # build tidy
        out = pd.DataFrame({
            "station": df["col0"].astype(str).str.strip().str.title(),
            "annual_entries_exits": df[total_col]
        })
 
        # drop junk rows (totals/headers)
        out = out.replace({"": np.nan, "nan": np.nan})
        out = out[~out["station"].str.contains(r"^total\b|^counts\b|^grand\b", case=False, na=False)]
        out = out.dropna(subset=["station", "annual_entries_exits"])
 
        # millions -> absolute (heuristic)
        if out["annual_entries_exits"].max(skipna=True) < 100_000:
            out["annual_entries_exits"] = out["annual_entries_exits"] * 1_000_000
 
        # daily average
        out["daily_avg_passengers"] = out["annual_entries_exits"] / 365.0
 
        # year from sheet name
        m = re.search(r"(20\d{2})", s)
        out["year"] = int(m.group(1)) if m else np.nan
 
        frames.append(out[["year","station","annual_entries_exits","daily_avg_passengers"]])
 
    if not frames:
        raise RuntimeError("Station workbook: produced no rows. Check file path or sheet names.")
 
    station_flow = pd.concat(frames, ignore_index=True).drop_duplicates()
    return station_flow
 
station_flow = clean_station_workbook(RAW_STATION_XLSX)
station_flow.to_csv(OUT_STATION_FLOW, index=False)
print(f"Wrote {OUT_STATION_FLOW}  rows={len(station_flow)}")
display(station_flow.head(10))
 
 
# ==== B) CLEAN STATIONS REFERENCE CSV (1 CSV) ===============================
def clean_stations_csv(csv_path: str) -> pd.DataFrame:
    """
    Simple clean for the station reference:
    - Trim whitespace, Title-case station names
    - Standardize column names
    - Coerce lat/lon if present
    - Drop duplicates
    """
    df = pd.read_csv(csv_path)
    # normalize headers
    df.columns = [re.sub(r"_+", "_", re.sub(r"[^0-9a-zA-Z]+", "_", c)).strip("_").lower() for c in df.columns]
 
    # pick common columns if present
    rename_map = {}
    for cand in ["station", "name", "station_name"]:
        if cand in df.columns:
            rename_map[cand] = "station"
            break
    for cand in ["line", "lines"]:
        if cand in df.columns:
            rename_map[cand] = "line"
            break
    for cand in ["latitude","lat"]:
        if cand in df.columns:
            rename_map[cand] = "lat"
            break
    for cand in ["longitude","lon","lng","long"]:
        if cand in df.columns:
            rename_map[cand] = "lon"
            break
    if rename_map:
        df = df.rename(columns=rename_map)
 
    if "station" in df.columns:
        df["station"] = df["station"].astype(str).str.strip().str.title()
    if "line" in df.columns:
        df["line"] = df["line"].astype(str).str.strip().str.title()
 
    # lat/lon if present
    for c in ["lat","lon"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
 
    # keep only useful columns if present
    keep = [c for c in ["station","line","lat","lon","borough","zone"] if c in df.columns]
    if keep:
        df = df[keep]
 
    df = df.drop_duplicates().dropna(how="all")
    return df
 
stations_clean = clean_stations_csv(RAW_STATIONS_CSV)
stations_clean.to_csv(OUT_STATIONS_CLEAN, index=False)
print(f"Wrote {OUT_STATIONS_CLEAN}  rows={len(stations_clean)}")
display(stations_clean.head(10))
 
 
# ==== C) CLEAN TUBE PERFORMANCE WORKBOOK (1 CSV) ============================
def clean_tube_performance(xlsx_path: str) -> pd.DataFrame:
    """
    Minimal, forgiving:
    - For each sheet: drop empty rows/cols
    - Choose a 'line' column (first non-numeric or header containing 'line')
    - Choose a 'date' column if available (period_start/date OR year+month), else leave NaT
    - Choose a 'metric' column: prefer EJT/LCH keywords else numeric with highest variance
    - Return one simple CSV: line, date, metric
    """
    xl = pd.ExcelFile(xlsx_path)
    frames = []
    for s in xl.sheet_names:
        df = xl.parse(s, header=0).dropna(how="all").dropna(axis=1, how="all")
        if df.shape[1] < 2:
            continue
 
        # normalize headers
        cols_norm = [re.sub(r"\s+", " ", str(c).strip().lower()) for c in df.columns]
        cmap = {cn: c for cn, c in zip(cols_norm, df.columns)}
 
        # line column
        line_col = None
        for key in ["line", "line name", "route"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                line_col = hit[0]; break
        if line_col is None:
            obj = df.select_dtypes(exclude=[np.number]).columns
            if len(obj) == 0: 
                continue
            line_col = obj[0]
 
        # date column (optional)
        date_series = None
        for key in ["period start","start date","date"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                date_series = pd.to_datetime(df[hit[0]], errors="coerce"); break
        if date_series is None and ("year" in cmap and "month" in cmap):
            y = pd.to_numeric(df[cmap["year"]], errors="coerce")
            m = pd.to_numeric(df[cmap["month"]], errors="coerce")
            date_series = pd.to_datetime(dict(year=y, month=m, day=1), errors="coerce")
 
        # metric column
        metric_col = None
        for key in ["excess journey time","ejt","lost customer hours","lch"]:
            hit = [cmap[k] for k in list(cmap) if key in k]
            if hit:
                metric_col = hit[0]; break
        if metric_col is None:
            num = df.select_dtypes(include=[np.number]).columns
            if len(num) == 0: 
                continue
            var = pd.Series({c: pd.to_numeric(df[c], errors="coerce").var(skipna=True) for c in num})
            metric_col = var.sort_values(ascending=False).index[0]
 
        tidy = pd.DataFrame({
            "line": df[line_col].astype(str).str.replace(" line","", case=False).str.title().str.strip(),
            "metric": pd.to_numeric(df[metric_col], errors="coerce")
        })
        if date_series is not None:
            tidy["date"] = pd.to_datetime(date_series, errors="coerce").dt.to_period("M").dt.to_timestamp()
        else:
            tidy["date"] = pd.NaT
 
        tidy = tidy.dropna(subset=["line"]).drop_duplicates()
        frames.append(tidy[["line","date","metric"]])
 
    if not frames:
        raise RuntimeError("Performance workbook: produced no rows. Check file path or sheet contents.")
    merged = pd.concat(frames, ignore_index=True)
    # collapse duplicates on same (line,date) by mean
    merged["date"] = pd.to_datetime(merged["date"], errors="coerce")
    merged = (merged
              .groupby(["line","date"], as_index=False)
              .agg(metric=("metric","mean")))
    return merged
 
perf_clean = clean_tube_performance(RAW_PERF_XLSX)
perf_clean.to_csv(OUT_PERF_CLEAN, index=False)
print(f"Wrote {OUT_PERF_CLEAN}  rows={len(perf_clean)}")
display(perf_clean.head(10))

Wrote data/processed/station_flow_clean_basic.csv  rows=2968


,year,station,annual_entries_exits,daily_avg_passengers
0,2017,Nan,253.0,0.693151
1,2017,625,149150.0,408.630137
2,2017,747,147574.0,404.312329
3,2017,669,121364.0,332.504110
4,2017,741,121018.0,331.556164
5,2017,635,113606.0,311.249315
6,2017,634,112437.0,308.046575
7,2017,719,93742.0,256.827397
8,2017,513,112547.0,308.347945
9,2017,852,86663.0,237.432877


Wrote data/processed/TfL_stations_clean.csv  rows=436


,station,line
0,Acton Town,"District, Piccadilly"
1,Barbican,"Circle, Hammersmith & City, Metropolitan"
2,Aldgate,"Circle, Metropolitan"
3,Aldgate East,"District, Hammersmith & City"
4,Alperton,Piccadilly
5,Amersham,Metropolitan
6,Angel,Northern
7,Archway,Northern
8,Arnos Grove,Piccadilly
9,Arsenal,Piccadilly


Wrote data/processed/tube_performance_clean_basic.csv  rows=0


,line,date,metric


In [54]:
####paths and imports
import pandas as pd
import numpy as np
from pathlib import Path
import re
import matplotlib.pyplot as plt
 
RAW_DIR = Path("data/processed")  # where your new cleaned CSVs were saved
 
STATION_FLOW_CSV   = RAW_DIR / "station_flow_clean_basic.csv"
STATIONS_CLEAN_CSV = RAW_DIR / "TfL_stations_clean.csv"
PERF_CLEAN_CSV     = RAW_DIR / "tube_performance_clean_basic.csv"
 
OUT_DIR = Path("data/processed")
OUT_DIR.mkdir(parents=True, exist_ok=True)

In [55]:
######load and sanity check
station_flow   = pd.read_csv(STATION_FLOW_CSV)
stations_ref   = pd.read_csv(STATIONS_CLEAN_CSV)
perf_clean     = pd.read_csv(PERF_CLEAN_CSV, parse_dates=["date"])
 
print("station_flow:", station_flow.shape, station_flow.columns.tolist())
print("stations_ref:", stations_ref.shape, stations_ref.columns.tolist())
print("perf_clean:",   perf_clean.shape,   perf_clean.columns.tolist())
 
display(station_flow.head(3))
display(stations_ref.head(3))
display(perf_clean.head(3))

station_flow: (2968, 4) ['year', 'station', 'annual_entries_exits', 'daily_avg_passengers']
stations_ref: (436, 2) ['station', 'line']
perf_clean: (0, 3) ['line', 'date', 'metric']


,year,station,annual_entries_exits,daily_avg_passengers
0,2017,Nan,253.0,0.693151
1,2017,625,149150.0,408.630137
2,2017,747,147574.0,404.312329


,station,line
0,Acton Town,"District, Piccadilly"
1,Barbican,"Circle, Hammersmith & City, Metropolitan"
2,Aldgate,"Circle, Metropolitan"


,line,date,metric


In [58]:
######## station/ line mapping
# Normalise names
station_flow["station"] = station_flow["station"].astype(str).str.strip().str.title()
if "line" in stations_ref.columns:
    stations_ref["line"] = stations_ref["line"].astype(str)
stations_ref["station"] = stations_ref["station"].astype(str).str.strip().str.title()
 
# Split multi-line cells (e.g. "Circle / H&C") and explode
def split_lines(s):
    if pd.isna(s): return []
    parts = re.split(r"[;/,&+]| and ", str(s), flags=re.I)
    return [p.strip().title() for p in parts if p.strip()]

In [57]:
#### line x year flow:
# Join station_flow ↔ stations_ref_exploded to attach a line to each station
sf_with_line = (
    station_flow.merge(
        stations_ref_exploded[["station","line"]],
        on="station",
        how="left"
    )
)
 
# If some stations don’t have a line mapping, keep them flagged (optional)
missing_map = sf_with_line["line"].isna().mean()
print(f"Unmapped stations (no line): {missing_map:.1%}")
 
# Aggregate to line × year
line_year = (
    sf_with_line
      .dropna(subset=["year"])  # year inferred from sheet name earlier
      .groupby(["line","year"], as_index=False)
      .agg(
          annual_entries_exits=("annual_entries_exits","sum"),
          line_daily_avg=("daily_avg_passengers","sum")
      )
)
 
# Create a monthly proxy load by dividing the annual by 12 (baseline)
line_month_proxy = (
    line_year
      .assign(month=1)
      .assign(date=lambda d: pd.to_datetime(dict(year=d["year"], month=d["month"], day=1)))
      .assign(monthly_load=lambda d: d["annual_entries_exits"] / 12.0)
      .sort_values(["line","date"])
      .drop(columns=["month"])
)
 
display(line_year.head(10))
display(line_month_proxy.head(10))
 
# Save for reuse
line_year.to_csv(OUT_DIR / "line_year_flow.csv", index=False)
line_month_proxy.to_csv(OUT_DIR / "line_month_load_proxy.csv", index=False)

 
if "line" in stations_ref.columns:
    stations_ref["line_list"] = stations_ref["line"].apply(split_lines)
    stations_ref_exploded = (
        stations_ref
        .explode("line_list", ignore_index=True)
        .rename(columns={"line_list": "line"})
        .drop(columns=["line"], errors="ignore")
    )
else:
    # If your reference file doesn’t have a line column yet, keep just station (we can still do station-level later)
    stations_ref_exploded = stations_ref.copy()
    stations_ref_exploded["line"] = np.nan
 
display(stations_ref_exploded.head(10))

NameError: name 'stations_ref_exploded' is not defined